# Sudoku Solver

The purpose of this project is to learn some basic concepts of AI like **constraint propagation** and **search** solving a sudoku.

A sudoku consists of a 9x9 grid. The objective is to fill the grid with digits such as each row, each column and each of the 9 principal 3x3 squares contain all digits from 1 to 9.

Initially we have a partially completed grid. In order to solve the puzzle, we should fill all the missing numbers.

An example of an initial sudoku:

<img src="img/sudoku-easy.png" width="60%">

## 1. Sudoku Representation

![title](img/labels.png)

The sudoku would have two representations: a `string` and a `dictionary`.

The `string` representation is the result of concatenating all the rows from top to bottom. We would use a `.` as a placeholder for an empty box.

The unsolved puzzle at above will be written as:
`..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..`

And the solved puzzle will be written as:
`483921657967345821251876493548132976729564138136798245372689514814253769695417382`

The dictionary representation will work as follows.

We will represent a box using the letters `'ABCDEFGHI'` for the rows and the numbers `'123456789'` for the columns. Each box will have the format: <ROW_LETTER><COLUMN_NUMBER>. For example: `'A1'`, `A2`, ..., `I9`. The values will be the digit in each box or a `.`(if the value is missing).

In [1]:
from utils import display

### Encode the rows and columns

In [2]:
rows = 'ABCDEFGHI'
columns = '123456789'

def cross(a, b):
    """
    Concatenate all letters from string `a`
    with all letters from string `b`.
    """
    return [s + t for s in a for t in b]

# Create the labels of the boxes
boxes = cross(rows, columns)
print(boxes)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']


### Create the units

We need to create the units for each box. A unit is a set of boxes the belongs to the same row, column or 3x3 square.

For example for the box `A1`.

Row unit: ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']

Column unit: ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']

Square unit: ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']


In [3]:
row_units = [cross(row, columns) for row in rows]
column_units = [cross(rows, column) for column in columns]
square_units = [
    cross(rs, cs)
    for rs in ('ABC', 'DEF', 'GHI')
    for cs in ('123', '456', '789')
]
unitlist = row_units + column_units + square_units
units = dict((box, [unit for unit in unitlist if box in unit]) for box in boxes)

print(row_units[0])
print(column_units[0])
print(square_units[0])
print('Row units for A1: ', units['A1'][0])
print('Column units for A1: ', units['A1'][1])
print('Square units for A1: ', units['A1'][2])

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']
Row units for A1:  ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
Column units for A1:  ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
Square units for A1:  ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']


### Create the peers

A peer is a box that belongs to the same row, column or 3x3 square.

In [4]:
peers = dict((box, set(sum(units[box], [])) - set([box])) for box in boxes)
peers['A1']

{'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'B1',
 'B2',
 'B3',
 'C1',
 'C2',
 'C3',
 'D1',
 'E1',
 'F1',
 'G1',
 'H1',
 'I1'}

### Convert string representation to a dictionary form

In [5]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    return dict(zip(boxes, grid))

In [6]:
sudoku_string = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
sudoku_dict = grid_values(sudoku_string)
sudoku_dict

{'A1': '.',
 'A2': '.',
 'A3': '3',
 'A4': '.',
 'A5': '2',
 'A6': '.',
 'A7': '6',
 'A8': '.',
 'A9': '.',
 'B1': '9',
 'B2': '.',
 'B3': '.',
 'B4': '3',
 'B5': '.',
 'B6': '5',
 'B7': '.',
 'B8': '.',
 'B9': '1',
 'C1': '.',
 'C2': '.',
 'C3': '1',
 'C4': '8',
 'C5': '.',
 'C6': '6',
 'C7': '4',
 'C8': '.',
 'C9': '.',
 'D1': '.',
 'D2': '.',
 'D3': '8',
 'D4': '1',
 'D5': '.',
 'D6': '2',
 'D7': '9',
 'D8': '.',
 'D9': '.',
 'E1': '7',
 'E2': '.',
 'E3': '.',
 'E4': '.',
 'E5': '.',
 'E6': '.',
 'E7': '.',
 'E8': '.',
 'E9': '8',
 'F1': '.',
 'F2': '.',
 'F3': '6',
 'F4': '7',
 'F5': '.',
 'F6': '8',
 'F7': '2',
 'F8': '.',
 'F9': '.',
 'G1': '.',
 'G2': '.',
 'G3': '2',
 'G4': '6',
 'G5': '.',
 'G6': '9',
 'G7': '5',
 'G8': '.',
 'G9': '.',
 'H1': '8',
 'H2': '.',
 'H3': '.',
 'H4': '2',
 'H5': '.',
 'H6': '3',
 'H7': '.',
 'H8': '.',
 'H9': '9',
 'I1': '.',
 'I2': '.',
 'I3': '5',
 'I4': '.',
 'I5': '1',
 'I6': '.',
 'I7': '3',
 'I8': '.',
 'I9': '.'}

In [7]:
display(sudoku_dict)

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


## 2. Solving the puzzle

In order to solve the puzzle we would use many techniques and approaches.

### Elimination

Let's look at the box `B5`:

<img src="img/reduce-values.png" width="60%">

We can see that the only possible values for this box are **4** and **7**. All other values already appear in the same column, row, or 3 x 3 square.

For example: 2, 3, 5, 6 and 8 appear in the same square. 1 and 9 appear in the same row.

We need to do this process for every box in the puzzle. The resulting puzzle would be:

<img src="img/values-easy.png" width="60%">


We need to update our `grid_values` function to encode every empty box with `'123456789'` instead of `'.'`.

In [8]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    new_grid = ['123456789' if value == '.' else value for value in grid]
    return dict(zip(boxes, new_grid))

In [9]:
sudoku_string = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
sudoku_dict = grid_values(sudoku_string)
display(sudoku_dict)

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

Now, implement the function eliminate:

In [10]:
def eliminate(grid):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_boxes = [box for box in grid.keys() if len(grid[box]) == 1]
    for box in solved_boxes:
        digit = grid[box]
        for peer in peers[box]:
            grid[peer] = grid[peer].replace(digit, '')
    return grid

In [11]:
sudoku_string = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
sudoku_dict = eliminate(grid_values(sudoku_string))
display(sudoku_dict)

   45    4578    3   |   49     2     147  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |  459   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6     478     9   |   5     1478    47  
   8     1467    47  |   2     457     3   |   17    1467    9   
   46    4679    5   |   4      1      47  |   3    24678   2467 


### Only choice

<img src="img/highlighted-unit.png" width="60%">

Let's look at the square highlighted in red. We can see that the only possible value for the box `'A6'` is **1**. That's because the digit **1** should appear somewhere in the unit and the only possible box is `'A6'`.

In [12]:
def only_choice(grid):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    for unit in unitlist:
        for digit in '123456789':
            digit_places = [box for box in unit if digit in grid[box]]
            if len(digit_places) == 1:
                grid[digit_places[0]] = digit
    return grid

In [13]:
sudoku_string = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
sudoku_dict = only_choice(eliminate(grid_values(sudoku_string)))
display(sudoku_dict)

  45    8     3   |  9     2     1   |  6    5789   57  
  9     6     47  |  3     4     5   |  8    278    1   
  2    257    1   |  8     7     6   |  4   23579  2357 
------------------+------------------+------------------
 345   345    8   |  1    3456   2   |  9   34567 34567 
  7     2     9   |  5   34569   4   |  1   13456   8   
 1345 13459   6   |  7    3459   8   |  2    1345  345  
------------------+------------------+------------------
 134   1347   2   |  6     8     9   |  5    1478   47  
  8    1467   47  |  2     5     3   |  17    6     9   
  6     9     5   |  4     1     7   |  3     8     2   


## 3. Constraint Propagation

Constrain propagations is all about to use local constraints in a space to reduce the search space. In the case of of Sudoku, each box have some conditions that help us to reduce the search space. We used those conditions to implement the functions `eliminate` and `only_choices`.

Now we can combine both functions to try to solve the puzzle.

In [14]:
def reduce_puzzle(grid):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in grid.keys() if len(grid[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        grid = eliminate(grid)

        # Your code here: Use the Only Choice Strategy
        grid = only_choice(grid)

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in grid.keys() if len(grid[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in grid.keys() if len(grid[box]) == 0]):
            return False
    return grid

In [15]:
sudoku_string = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
sudoku_dict = reduce_puzzle(grid_values(sudoku_string))
display(sudoku_dict)

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


We finally solved our first sudoku :)

## 4. Search

First, let's try our previous algorithm to solve a harder sudoku puzzle:

<img src="img/harder-puzzle.png" width="60%">

Sadly, the algorithm couldn't solve it. We need to use other approach to solve the puzzle.

<img src="img/harder-sudoku-reduced.png" width="60%">

In [16]:
harder_sudoku_string = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
harder_sudoku_dict = grid_values(harder_sudoku_string)
display(reduce_puzzle(harder_sudoku_dict))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


### Search Tree

<img src="img/alphago-search-tree.png">

<center>Search used in Google's AlphaGo __[paper](https://storage.googleapis.com/deepmind-media/alphago/AlphaGoNaturePaper.pdf)__<center>

We need to generate a search tree like the example above. The box `'C1'` has four possibilities: 2, 6, 8, and 9. We can fille it in with a 2 and try to solve our puzzle. If we can't solve it, we will try with a 6, then with a 8, and then with a 9.

We will start with a box with a minimal number of possible values. We will create a search tree of possibilities and traverse it using DFS untils it finds a solution

In [17]:
def search(grid):
    """Using depth-first search and propagation, try all possible values."""
    
    # First, reduce the puzzle using the previous function
    grid = reduce_puzzle(grid)
    if grid is False:
        return False  # Failed earlier
    if all(len(grid[box]) == 1 for box in boxes):
        return grid  # Solved!
    
    # Choose one of the unfilled squares with the fewest possibilities
    _, box = min(
        (len(grid[box]), box)
        for box in boxes if len(grid[box]) > 1
    )

    # Now use recurrence to solve each one of the resulting sudokus
    for value in grid[box]:
        new_sudoku = grid.copy()
        new_sudoku[box] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [18]:
harder_sudoku_string = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
harder_sudoku_dict = grid_values(harder_sudoku_string)
display(search(harder_sudoku_dict))

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 


## 3. Additional Techniques

### Naked Twins

<img src="img/naked-twins.png" width="60%">

The highlighted boxes belong to the same unit(column), and both permit the values 2 and 3. We don't know their exact positions, but we know that the values 2 and 3 are locked in those two boxes, so we can eliminate both from their peers in the same unit.

<img src="img/naked-twins-2.png" width="60%">


In [19]:
def naked_twins(grid):
    """Eliminate values using the naked twins strategy.

    Parameters
    ----------
    grid(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the naked twins eliminated from peers

    Notes
    -----
    Your solution can either process all pairs of naked twins from the input once,
    or it can continue processing pairs of naked twins until there are no such
    pairs remaining -- the project assistant test suite will accept either
    convention. However, it will not accept code that does not process all pairs
    of naked twins from the original input. (For example, if you start processing
    pairs of twins and eliminate another pair of twins before the second pair
    is processed then your code will fail the PA test suite.)

    The first convention is preferred for consistency with the other strategies,
    and because it is simpler (since the reduce_puzzle function already calls this
    strategy repeatedly).
    """
    initial_pairs = [box for box in grid if len(grid[box]) == 2]
    
    # Iterate all boxes with two digits
    for box in initial_pairs:
        # Find the twins for this box
        for peers in units[box]:
            found_twin = False
            peers_to_update = set()
            for peer in peers:
                if peer == box:
                    continue
                if grid[box] == grid[peer]:
                    found_twin = True
                else:
                    peers_to_update.add(peer)
            
            # If a twin is found we update the rest of the boxes in that unit
            if found_twin:
                for digit in grid[box]:
                    for peer in peers_to_update:
                        grid[peer] = grid[peer].replace(digit, '')
    return grid

In [20]:
before_naked_twins = {
    'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8',
    'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8',
    'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5',
    'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27',
    'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
    'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2',
    'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6',
    'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9',
    'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27',
    'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279',
    'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}
display(before_naked_twins)

  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1   2379| 237  347  279 |  34   8    6  
  8    37  379 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  


In [21]:
after_naked_twins = naked_twins(before_naked_twins)
display(after_naked_twins)

  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1    79 | 237  347  279 |  34   8    6  
  8    37   79 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  


### Diagonal Sudoku

A diagonal sudoku it's a variant of the regular sudoku in which each of the two main diagonals form a unit.

<img src="img/diagonal-sudoku.png" width="60%">


We will modify our previous units to handle this specific case.

In [22]:
diagonal_units = [
    [s + t for s, t in zip(rows, columns) ],
    [s + t for s, t in zip(rows, reversed(columns))]
]
unitlist = row_units + column_units + square_units + diagonal_units
peers = dict((box, set(sum(units[box], [])) - set([box])) for box in boxes)

In [23]:
diagonal_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
grid_dict = grid_values(diagonal_grid)
display(grid_dict)

    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678

In [24]:
display(search(grid_dict))

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


## TODO

- We can add the naked twins strategy inside our `reduce_puzzle` function.
- We can modify the function `display` to show the sudoku in an image.
- We can implement the game. The game can show sugestions using the strategies we implemented.